### Today we are going to perform the simple classification of the amazon reviews' sentiment.

### Please, download the dataset amazon_baby.csv.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.linear_model import LogisticRegression

def remove_punctuation(text):
    import string
    translator = str.maketrans('', '', string.punctuation)
    # translate() method returns a copy of the string in
    # which all characters have been translated using table
    # (constructed with the maketrans() function in the string module)
    return text.translate(translator)

# from google.colab import files
# uploaded = files.upload()
# import io
# baby_df = pd.read_csv(io.BytesIO(uploaded['amazon_baby.csv']))
baby_df = pd.read_csv('amazon_baby.csv')
baby_df.head()

,name,review,rating
0,Planetwise Flannel Wipes,"These flannel wipes are OK, but in my opinion ...",3
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5
3,Stop Pacifier Sucking without tears with Thumb...,This is a product well worth the purchase. I ...,5
4,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5


## Exercise 1 (data preparation)
a) Remove punctuation from reviews using the given function.   
b) Replace all missing (nan) revies with empty "" string.  
c) Drop all the entries with rating = 3, as they have neutral sentiment.   
d) Set all positive ($\geq$4) ratings to 1 and negative($\leq$2) to -1.

In [2]:
#a)
baby_df['review'] = remove_punctuation(baby_df['review'].str)
#short test: 
baby_df["review"][4] == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'
remove_punctuation(baby_df["review"][4]) == 'All of my kids have cried nonstop when I tried to ween them off their pacifier until I found Thumbuddy To Loves Binky Fairy Puppet  It is an easy way to work with your kids to allow them to understand where their pacifier is going and help them part from itThis is a must buy book and a great gift for expecting parents  You will save them soo many headachesThanks for this book  You all rock'

True

In [3]:
#b)
# baby_df['rating'].value_counts()
baby_df['review'] = baby_df['review'].replace(np.nan, '', regex=True)
#short test:
baby_df["review"][38] == baby_df["review"][38]


True

In [4]:
#c)
baby_df = baby_df[baby_df.rating != 3]
#short test:
sum(baby_df["rating"] == 3)

0

In [5]:
#d) 
baby_df.loc[baby_df['rating'] <= 2, 'rating'] = -1
baby_df.loc[baby_df['rating'] >= 4, 'rating'] = 1
#short test:
sum(baby_df["rating"]**2 != 1)

0

## CountVectorizer
In order to analyze strings, we need to assign them numerical values. We will use one of the simplest string representation, which transforms strings into the $n$ dimensional vectors. The number of dimensions will be the size of our dictionary, and then the values of the vector will represent the number of appereances of the given word in the sentence.

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
reviews_train_example = ["We like apples",
                   "We hate oranges",
                   "I adore bananas",
                   "We like like apples and oranges",
                   "They dislike bananas"]

X_train_example = vectorizer.fit_transform(reviews_train_example)

print(vectorizer.get_feature_names())
print(X_train_example.todense())



['adore', 'and', 'apples', 'bananas', 'dislike', 'hate', 'like', 'oranges', 'they', 'we']
[[0 0 1 0 0 0 1 0 0 1]
 [0 0 0 0 0 1 0 1 0 1]
 [1 0 0 1 0 0 0 0 0 0]
 [0 1 1 0 0 0 2 1 0 1]
 [0 0 0 1 1 0 0 0 1 0]]


In [7]:
reviews_test_example = ["They like bananas",
                   "We hate oranges bananas and apples",
                   "We love bananas"] #New word!

X_test_example = vectorizer.transform(reviews_test_example)

print(X_test_example.todense())

[[0 0 0 1 0 0 1 0 1 0]
 [0 1 1 1 0 1 0 1 0 1]
 [0 0 0 1 0 0 0 0 0 1]]


We should acknowledge few facts. Firstly, CountVectorizer does not take order into account. Secondly, it ignores one-letter words (this can be changed during initialization). Finally, for test values, CountVectorizer ignores words which are not in it's dictionary.

## Exercise 2 
a) Split dataset into training and test sets.     
b) Transform reviews into vectors using CountVectorizer. 

In [101]:
#a)
from sklearn.model_selection import train_test_split
train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(baby_df['review'], baby_df['rating'])

In [102]:
#b)
vectorizer = CountVectorizer()
vectorized_train_reviews = vectorizer.fit_transform(train_reviews)
vectorized_test_reviews = vectorizer.transform(test_reviews)

## Exercise 3 
a) Train LogisticRegression model on training data (reviews processed with CountVectorizer, ratings as they were).   
b) Print 10 most positive and 10 most negative words.

In [103]:
#a)
model = LogisticRegression()
model.fit(vectorized_train_reviews, train_ratings)

c:\users\datacomplex\appdata\local\programs\python\python39\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [104]:
#b)
words = sorted(list(zip(model.coef_[0], vectorizer.get_feature_names())), key=lambda x: x[0])
print('Top 10 most positive words:', [word[1] for word in words[-10:]])
print('Top 10 most negative words:', [word[1] for word in words[:10]])
#hint: model.coef_, vectorizer.get_feature_names()

Top 10 most positive words: ['pleased', 'highly', 'satisfied', 'amazing', 'awesome', 'loves', 'complaint', 'perfect', 'excellent', 'worry']
Top 10 most negative words: ['disappointing', 'worst', 'concept', 'disappointed', 'poorly', 'useless', 'unusable', 'returning', 'worthless', 'terrible']


## Exercise 4 
a) Predict the sentiment of test data reviews.   
b) Predict the sentiment of test data reviews in terms of probability.   
c) Find five most positive and most negative reviews.   
d) Calculate the accuracy of predictions.

In [105]:
#a)
import time
start = time.time()
rating_predictions = model.predict(vectorized_test_reviews)
rating_preds_time = time.time() - start
print(rating_predictions)

[ 1 -1  1 ...  1  1  1]


In [106]:
#b)
# The first index refers to the probability that the data belong to class -1,
# and the second refers to the probability that the data belong to class 1.
rating_proba_predictions = model.predict_proba(vectorized_test_reviews)
print(rating_proba_predictions)
#hint: model.predict_proba()

[[3.09059218e-03 9.96909408e-01]
 [9.99996821e-01 3.17945127e-06]
 [1.68394118e-03 9.98316059e-01]
 ...
 [2.37320137e-03 9.97626799e-01]
 [8.18224235e-05 9.99918178e-01]
 [6.30142707e-03 9.93698573e-01]]


In [107]:
#c) 
#hint: use the results of b)
reviews = sorted(list(zip(rating_proba_predictions, test_reviews)), key=lambda x: x[0][1])
top_5_positive = [review[1] for review in reviews[-5:]]
top_5_negative = [review[1] for review in reviews[:5]]
print('Top 5 most positive reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_positive[i]))
print('Top 5 most negative reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_negative[i]))

Top 5 most positive reviews:

1. I was very excited when I heard Chicco was finally coming out with a double stroller I was initially against any double stroller I just didnt want to mess with it But when our daughter was 25 years and our son was four months we were taking a month long trip abroad and I knew we wouldnt be able to corral both kids without one The Cortina Together worked wonderfully for what we needed We already had a Chicco car seat and Trevi so we knew we liked Chicco And I wanted both kids to be able to sit comfortably I knew a sit and stand would be lighter but didnt want my twoyearold to have to stand or sit on an uncomfortable tiny seat all the time After hauling this thing all over Europe here is why I love this strollerYou can easily snap the Chicco car seat into it I love that you can place it in the front seat or back seat When we want more control of our toddler we place her in the back and baby in the front But when we are just enjoying a walk she loves sitti

In [108]:
#d) 
from sklearn.metrics import accuracy_score
acc_score_rating_preds = accuracy_score(test_ratings, rating_predictions)
print(acc_score_rating_preds)

0.9312991748224909


## Exercise 5
In this exercise we will limit the dictionary of CountVectorizer to the set of significant words, defined below.


a) Redo exercises 2-5 using limited dictionary.   
b) Check the impact of all the words from the dictionary.   
c) Compare accuracy of predictions and the time of evaluation.

In [109]:
significant_words = ['love','great','easy','old','little','perfect','loves','well','able','car','broke','less','even','waste','disappointed','work','product','money','would','return']

In [110]:
#a)
from sklearn.model_selection import train_test_split
train_reviews, test_reviews, train_ratings, test_ratings = train_test_split(baby_df['review'], baby_df['rating'])

# #b)
vectorizer = CountVectorizer(vocabulary=significant_words)
vectorized_train_reviews = vectorizer.fit_transform(train_reviews) # instead of train_reviews
vectorized_test_reviews = vectorizer.transform(test_reviews)

# #a)
model = LogisticRegression()
model.fit(vectorized_train_reviews, train_ratings)

# #b)
words = sorted(list(zip(model.coef_[0], vectorizer.get_feature_names())), key=lambda x: x[0])
print('Top 10 most positive words:', [word[1] for word in words[-10:]])
print('Top 10 most negative words:', [word[1] for word in words[:10]])

# #a)
start = time.time()
dict_rating_predictions = model.predict(vectorized_test_reviews)
print(dict_rating_predictions)
rating_dict_preds_time = time.time() - start

# #b)
rating_proba_predictions = model.predict_proba(vectorized_test_reviews)
print(rating_proba_predictions)

# #c) 
reviews = sorted(list(zip(rating_proba_predictions, test_reviews)), key=lambda x: x[0][1])
top_5_positive = [review[1] for review in reviews[-5:]]
top_5_negative = [review[1] for review in reviews[:5]]
print('Top 5 most positive reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_positive[i]))
print('Top 5 most negative reviews:\n')
for i in range(5):
    print('{}. {}\n'.format(i + 1, top_5_negative[i]))

# #d) 
from sklearn.metrics import accuracy_score

acc_score_dict_rating_preds = accuracy_score(test_ratings, dict_rating_predictions)

Top 10 most positive words: ['car', 'old', 'able', 'little', 'well', 'great', 'easy', 'love', 'perfect', 'loves']
Top 10 most negative words: ['disappointed', 'waste', 'return', 'broke', 'money', 'work', 'even', 'would', 'product', 'less']
[1 1 1 ... 1 1 1]
[[0.12144008 0.87855992]
 [0.08486193 0.91513807]
 [0.02516944 0.97483056]
 ...
 [0.14467686 0.85532314]
 [0.49162951 0.50837049]
 [0.11422531 0.88577469]]
Top 5 most positive reviews:

1. I literally have owned almost every stroller No I am not a shopaholic I just think and know that there is no perfect stroller so as needs changed strollers were addedI now have 3 children and one very messy garage  Heres the listPeg Perego Venezia nice ride too big too heavyAprica great shade tough to maneuver sometimes wont close STRESSSnap N go miracle stroller for the first 12 months provided baby stays with in the weight rangeMaclaren Triumph awesome umbrella stroller no snack tray for parent or childCountless cheap umbrella stroller I toss an

In [111]:
#b) Check the impact of all the words from the dictionary.   
from math import fabs
max_impact = max(max(model.coef_[0]), fabs(min(model.coef_[0])))
print('Maximum relative impact = {0:.5f}'.format(max_impact))
dict_words_impact = [val for val in words if any(s in val for s in significant_words)]
for word_impact in dict_words_impact:
    print('Word {0} has impact {1:.5f}'.format(word_impact[1], fabs(word_impact[0])))

Maximum relative impact = 2.38399
Word disappointed has impact 2.38399
Word waste has impact 2.02241
Word return has impact 1.99869
Word broke has impact 1.67000
Word money has impact 0.91777
Word work has impact 0.61503
Word even has impact 0.51331
Word would has impact 0.34165
Word product has impact 0.32388
Word less has impact 0.18445
Word car has impact 0.06887
Word old has impact 0.07266
Word able has impact 0.22872
Word little has impact 0.48407
Word well has impact 0.49927
Word great has impact 0.93983
Word easy has impact 1.17873
Word love has impact 1.36060
Word perfect has impact 1.47114
Word loves has impact 1.72346


In [112]:
#c) Compare accuracy of predictions and the time of evaluation.
print('Using all the words available give better results by {0:.2f}%.'
      .format((acc_score_rating_preds-acc_score_dict_rating_preds)*100))
print('Time of evaluation of all the words: {0:.5f}ms, of limited dictionary: {1:.5f}ms'.format(rating_preds_time * 1000, rating_dict_preds_time * 1000))
#hint: %time, %timeit

Using all the words available give better results by 6.16%.
Time of evaluation of all the words: 23.93389ms, of limited dictionary: 0.00000ms
